In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from PIL import Image
import numpy as np
image = Image.open("img.png").convert('L')
img_np = np.array(image)
img_np.shape

(1025, 800)

In [3]:
from UNET import DoubleConv

x = torch.from_numpy(img_np).float()
x = torch.unsqueeze(x, dim=1).float()
x = x.permute(1,0,2).unsqueeze(0)
# print(x.shape)
doubleconv = DoubleConv(in_channels=1, out_channels=64)
doubleconv(x).shape
x.size()

torch.Size([1, 1, 1025, 800])

In [6]:
from UNET import UNET

model = UNET(n_channels=1, n_class=1)

In [8]:
output = model(x)

In [9]:
output.shape

torch.Size([1, 1, 1025, 800])

In [ ]:
import torch.nn.functional as F
y = torch.randn(1,1,1025,800)
F.binary_cross_entropy_with_logits(x, y, reduction='mean'), y.shape

(tensor(105.0129), torch.Size([1, 1, 1025, 800]))

RuntimeError: all elements of input should be between 0 and 1

In [75]:
import pathlib
from data import P3MDataset
from pathlib import Path
full_dataset = P3MDataset(
    input_path=Path('P3M-500-NP/original_image'),
    label_path=Path('P3M-500-NP/mask')
)

In [78]:
from torch.utils.data import random_split
total_size = len(full_dataset)
train_size = int(0.9 * total_size)
test_size = total_size - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True,
    num_workers=4
)
test_loader = DataLoader(
    dataset=test_dataset,
    shuffle=False,
    batch_size=12,
    num_workers=2
)

In [62]:
for inputs, labels in dataloader:
    print(f"input shape: {inputs.shape}")
    print(f"labels_shape: {labels.shape}")
    break

input shape: torch.Size([1, 1232, 1080, 3])
labels_shape: torch.Size([1, 1232, 1080, 3])


In [71]:
new_mask_folder = Path('P3M-500-NP/new_mask')
mask_folder = Path('P3M-500-NP/mask')
for mask_path in mask_folder.iterdir():
    mask_np = np.array(Image.open(mask_path))
    mask_np[mask_np > 0] = 255
    # print(new_mask_folder/f"{mask_path.stem}/.png")
    Image.fromarray(mask_np).save(new_mask_folder/f"{mask_path.stem}.png")

In [5]:
import os
from PIL import Image

data_dir = 'P3M-500-NP/original_image/'
invalid_files = []

print("Đang kiểm tra dữ liệu...")
for filename in os.listdir(data_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        path = os.path.join(data_dir, filename)
        try:
            with Image.open(path) as img:
                img.verify() # Kiểm tra xem file có bị hỏng không
        except (IOError, SyntaxError, Image.UnidentifiedImageError):
            print(f"File lỗi: {path}")
            os.remove(path)
            invalid_files.append(path)

# Tùy chọn: Xóa hoặc di chuyển file lỗi
# for f in invalid_files: os.remove(f)

Đang kiểm tra dữ liệu...


In [4]:
invalid_files

['P3M-500-NP/original_image/p_669f7b92.jpg']

In [19]:
import numpy as np
indices = list(range(5))
np.random.shuffle(indices)
indices


[4, 0, 2, 3, 1]

In [26]:
import torch
x1 = torch.tensor([1,2])
x2 = torch.tensor([3,4])
torch.cat((x1, x2), dim = 0), torch.stack((x1, x2), dim=0), torch.stack((x1, x2), dim=1)

(tensor([1, 2, 3, 4]),
 tensor([[1, 2],
         [3, 4]]),
 tensor([[1, 3],
         [2, 4]]))